In [2]:
data = require 'data'

In [3]:
dir = '/home/arun/data/seqsimple2/'

In [4]:
X,Y = data.loadXY(dir)
data.convert(Y)

In [5]:
function repl(N)
    local net = nn.ConcatTable()
    for i=1,N do 
        net:add(nn.Identity())
    end
    return net
end

In [6]:
require 'nn'
require 'rnn'
net = nn.Sequential()
net:add(nn.Reshape(1,50,200))
net:add(nn.SpatialConvolution(1,64,3,3,1,1,1,1))
net:add(nn.SpatialBatchNormalization(64,1e-3))
net:add(nn.ReLU(true))
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
net:add(nn.SpatialBatchNormalization(64,1e-3))
net:add(nn.ReLU(true))
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
net:add(nn.SpatialBatchNormalization(64,1e-3))
net:add(nn.ReLU(true))
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
net:add(nn.SpatialBatchNormalization(64,1e-3))
net:add(nn.ReLU(true))
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
net:add(nn.SpatialBatchNormalization(64,1e-3))
net:add(nn.ReLU(true))
net:add(nn.SpatialMaxPooling(2,2,2,2))
--net:add(nn.View(64*1*6))
net:add(nn.Reshape(8,48))
net:add(nn.SplitTable(2,3))
enc = nn.Sequential()
enc:add(net)
enc:add(nn.Sequencer(nn.LSTM(48, 48)))
enc:add(nn.SelectTable(-1))
dec = nn.Sequential()
dec:add(enc)
dec:add(repl(7))

mlp = nn.Sequential()
       :add(nn.LSTM(48, 48))
       :add(nn.Linear(48, 36))
       :add(nn.LogSoftMax())

dec:add(nn.Sequencer(mlp))
--net:add(repl(7))

In [7]:
require 'cunn';
require 'nn';
require 'rnn';
dec = torch.load('seq88.t7')
dec = dec:cuda()

In [8]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion())
criterion = criterion:cuda()

In [9]:
rutil = require 'rutil'

In [10]:
Xt,Yt,Xv,Yv = data.split(X,Y,1000)

In [11]:
tnet = nn.SplitTable(2,2):cuda()

In [12]:
rutil.valid(dec,Xv,Yv,512,tnet)

95.6	{
  1 : 99.3
  2 : 94.6
  3 : 91.9
  4 : 93.1
  5 : 93.4
  6 : 97.4
  7 : 99.5
}


In [33]:

function rutil.kfacc(outputs,targets)
    local Y,y = nil,nil;
    local N = outputs[1]:size(1)
    local C = outputs[1]:size(2)
    for k=1,#outputs do 
        Y = Y and torch.cat(Y,outputs[k]:reshape(N,1,C),2) or outputs[k]:reshape(N,1,C)
        y = y and torch.cat(y,targets[k]:reshape(N,1),2) or targets[k]:reshape(N,1)
    end
    local t,idx = Y:max(3)
    return idx:squeeze():eq(y):sum(2):eq(#outputs):sum()
end

function rutil.kvalid(rnn,Xv,Yv,batchSize,tnet)
    local batchSize = batchSize or 16
    local acc = 0
    local acci = {}
    local Nv = Xv:size(1)
    rnn:evaluate()
    for i=1,Nv,batchSize do 
        xlua.progress(i/batchSize, Nv/batchSize)
        local j = math.min(Nv,i+batchSize-1)
        local Xb = Xv[{{i,j}}]:cuda()            
        local Yb = Yv[{{i,j}}]:cuda()
        local inputs = Xb
        local targets = tnet:forward(Yb)
        local outputs = rnn:forward(inputs)
        local aa,ai = rutil.kfacc2(outputs,targets)
        acc = acc + aa
        rnn:forget()
    end
    return (acc*100)/Nv,acci
end


In [16]:
rutil.train(dec,criterion,Xt,Yt,Xv,Yv,8,64,tnet,0.01)

11:26:40  IST	
1	8	


Progress: 15.015625 / 453.125	


Progress: 60.015625 / 453.125	


Progress: 106.015625 / 453.125	


Progress: 152.015625 / 453.125	


...

Progress: 349.015625 / 453.125	


Progress: 395.015625 / 453.125	


Progress: 441.015625 / 453.125	


loss	49.389657061015	
train	98.445320197044	


v	97.214285714286	maxv	97.528571428571	
{
  1 : 99.6
  2 : 96.8
  3 : 94.1
  4 : 95.4
  5 : 96.4
  6 : 98.6
  7 : 99.6
}
11:47:05  IST	
97.528571428571	



In [17]:
rutil.train(dec,criterion,Xt,Yt,Xv,Yv,8,64,tnet,0.01)

11:49:09  IST	
1	8	


Progress: 28.015625 / 453.125	


Progress: 73.015625 / 453.125	


Progress: 118.015625 / 453.125	


Progress: 163.015625 / 453.125	


...

Progress: 356.015625 / 453.125	


Progress: 401.015625 / 453.125	


Progress: 446.015625 / 453.125	


loss	45.20982479517	
train	98.778325123153	


v	97.085714285714	maxv	97.171428571429	
{
  1 : 99.6
  2 : 96.7
  3 : 94.1
  4 : 95.3
  5 : 96
  6 : 98.2
  7 : 99.7
}
12:09:39  IST	
97.171428571429	



In [ ]:
torch.save('seq88.t7',dec)

In [30]:
rutil.kvalid(dec,Xv,Yv,64,tnet)

44.1	{
}


In [18]:
rutil.train(dec,criterion,Xt,Yt,Xv,Yv,8,64,tnet,0.1)

12:09:39  IST	
1	8	


Progress: 32.015625 / 453.125	


Progress: 76.015625 / 453.125	


Progress: 120.015625 / 453.125	


Progress: 164.015625 / 453.125	


...

Progress: 331.015625 / 453.125	


Progress: 377.015625 / 453.125	


Progress: 423.015625 / 453.125	


loss	88.091715707311	
train	95.120689655172	


v	26.228571428571	maxv	83.571428571429	
{
  1 : 28.8
  2 : 34.2
  3 : 22.5
  4 : 12
  5 : 11.1
  6 : 23.2
  7 : 51.8
}
12:30:18  IST	
83.571428571429	



In [22]:
rutil.train(dec,criterion,Xt,Yt,Xv,Yv,1,64,tnet,0.1)

12:34:52  IST	
1	1	


Progress: 44.015625 / 453.125	


Progress: 89.015625 / 453.125	


Progress: 135.015625 / 453.125	


Progress: 180.015625 / 453.125	


...

Progress: 363.015625 / 453.125	


Progress: 408.015625 / 453.125	


loss	72.135594595756	
train	96.106896551724	


Progress: 0.0625 / 62.5	


v	88.285714285714	maxv	88.285714285714	
{
  1 : 98
  2 : 86.3
  3 : 80.2
  4 : 80.6
  5 : 84.1
  6 : 91.1
  7 : 97.7
}
12:37:25  IST	
88.285714285714	



In [50]:
function rutil.check(y,Y)
    --print(y,Y)
    for i=1,y:nElement() do
        if(Y[i]==1) then return true end
        if(y[i]~=Y[i]) then return false end
    end
    return true
end
function rutil.kfacc2(outputs,targets)
    local Y,y = nil,nil;
    local N = outputs[1]:size(1)
    local C = outputs[1]:size(2)
    for k=1,#outputs do 
        Y = Y and torch.cat(Y,outputs[k]:reshape(N,1,C),2) or outputs[k]:reshape(N,1,C)
        y = y and torch.cat(y,targets[k]:reshape(N,1),2) or targets[k]:reshape(N,1)
    end
    local t,idx = Y:max(3)
    idx = idx:squeeze()
    local acc = 0
    for i=1,y:size(1) do
        if(rutil.check(idx[i],y[i])) then 
            acc = acc + 1
            --print('true')
        else
            --print('false')
        end
    end
    return acc
end


In [51]:
rutil.kvalid(dec,Xv,Yv,64,tnet)

44.1	{
}


In [52]:
torch.save('seq44.t7',dec)